In [ ]:
!pip install natsort

In [ ]:
!pip install scipy==1.12.0
!pip install scikit-plot==0.3.7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import joblib
import sys
from natsort import natsorted
sys.modules['sklearn.externals.joblib'] = joblib
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
########################################################
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Flatten, \
    LSTM, Dense, Dropout, Embedding, Bidirectional, GRU,Flatten, Activation,\
    BatchNormalization, UpSampling2D,Conv1D, Input, Concatenate,multiply, add,\
    AveragePooling1D, UpSampling1D,Lambda, Average, Dense, Flatten,GRU,Attention
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,\
    ReduceLROnPlateau, LearningRateScheduler, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Lambda, Conv2D, Activation,\
    BatchNormalization, UpSampling2D, multiply, add
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, optimizers
#from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import constraints
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
########################################################
from random import  randint
from random import choice
from random import uniform
from random import  randint
from random import choice
from sklearn.metrics import classification_report, confusion_matrix
from random import uniform
import scipy

In [ ]:
!mkdir defect_bearing_saved_pictures

mkdir: cannot create directory ‘defect_bearing_saved_pictures’: File exists


In [ ]:
!cp /content/drive/MyDrive/defect_bearing_saved_pictures/defect_bearing_saved_pictures1.zip /content/sample_data

In [ ]:

with zipfile.ZipFile("/content/sample_data/defect_bearing_saved_pictures1.zip","r") as zip_ref:
    zip_ref.extractall("/content/defect_bearing_saved_pictures")


In [ ]:

class reading_data_picture():
    def __init__(self, path, img_size_x, img_size_y):
        self.path = path
        self.img_size_x = img_size_x
        self.img_size_y = img_size_y
    def get_data(self):
        data = []
        picture_list = os.listdir(self.path)
        picture_list = natsorted(picture_list)
        for x in picture_list:
            picture_path = os.path.join(self.path,x)
            out = cv2.imread(picture_path)
            out_resize = cv2.resize(out,(self.img_size_x, self.img_size_y))
            data.append(out_resize)
        data_out = np.array(data) /255
        return data_out

In [ ]:
data_total  = reading_data_picture('/content/defect_bearing_saved_pictures', 224, 224)
data_picture = data_total.get_data()

In [ ]:
class reading_data():
  def __init__(self, path):
    self.path = path
  def reading_data(self):
    df = pd.read_csv(self.path)
    print(df.isnull().sum())
    if df.isnull().sum().all():
      df = df.fillna(df.iloc[0])
    else:
      df = df
    return df
  def visualization_pearson(self):
    df = pd.read_csv(self.path)
    corr =df.corr()
    return sns.heatmap(
        corr, annot=True,
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)
  def visualization_kendas(self):
    df = pd.read_csv(self.path)
    corr =df.corr(method='kendall')
    return sns.heatmap(
        corr, annot=True,
        center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)
  def visualization_spearman(self):
    df = pd.read_csv(self.path)
    corr =df.corr(method='spearman')
    return sns.heatmap(
        corr, annot=True,
        center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)



In [ ]:
data = reading_data("/content/bearing-failure.csv")
df = data.reading_data()
target = df['Label']

COMP_NAME                  0
Vel, Rms (RMS)             0
Acc, Rms (RMS)             0
Crest (RMS)                0
Kurt (RMS)                 0
Vel, Peak (RMS)            0
Vel, Peak to peak (RMS)    0
MP_LOC                     0
Label                      0
dtype: int64


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
traina , trainb, target_train, testa , testb, target_test =   data_picture[0:1100], np.array(df.iloc[0:1100]).reshape(1100,9,1), encoder.fit_transform(np.array(target.iloc[0:1100]).reshape(-1,1)), data_picture[1100:],np.array(df.iloc[1100:]).reshape(284, 9, 1) , encoder.fit_transform(np.array(target.iloc[1100:]).reshape(-1,1))
print("The training size for tabular is: ",trainb.shape)
print("The training size for picture data is: ",traina.shape)
print("The training target size for picture is: ",target_train.shape)
print("The testing size for tabular is: ",testb.shape )
print("The testing size for picture data is: ",testa.shape )
print("The testing target size for picture is: ",target_test.shape )

The training size for tabular is:  (1100, 9, 1)
The training size for picture data is:  (1100, 224, 224, 3)
The training target size for picture is:  (1100, 3)
The testing size for tabular is:  (284, 9, 1)
The testing size for picture data is:  (284, 224, 224, 3)
The testing target size for picture is:  (284, 3)


In [ ]:
MH_layer = MultiHeadAttention(num_heads=8, key_dim=4)
target = tf.keras.Input(shape=[200, 4])
source = tf.keras.Input(shape=[200, 4])

In [ ]:
MH_layer_EEG = MultiHeadAttention(num_heads=8, key_dim=4)
target_EEG = tf.keras.Input(shape=[21,21,2])
source_EEG = tf.keras.Input(shape=[25,25,1])

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
#time in hospital
class Net:
    def __init__(self):
        self.ep = randint(100, 200)               # epoch
        self.u1 = randint(800, 1682)
        self.d1 = choice([0.15, 0.20, 0.25])         #  dropout 1
        self.a1 = choice([('relu'),('elu'),('tanh'),('leaky_relu'),('selu'), ('swish')])                      # activation 1
        self.a2 = choice([('relu'),('elu'),('tanh'),('leaky_relu'),('selu'), ('swish')])                      # activation 2
        self.a3 = choice([('relu'),('elu'),('tanh'),('leaky_relu'),('selu'), ('swish')])                      # activation 3
        self.a4 = 'softmax'                                                                                   # activation
        self.pretrain =  choice([(tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')),
                                 (tf.keras.applications.ResNet50V2(input_shape=(224,224,3),include_top=False,weights='imagenet')),
                                 (tf.keras.applications.VGG19(input_shape=(224,224,3),include_top=False,weights='imagenet')),
                                 (tf.keras.applications.Xception(input_shape=(224,224,3),include_top=False,weights='imagenet')),
                                 (tf.keras.applications.DenseNet201(input_shape=(224,224,3),include_top=False,weights='imagenet'))])



        self.lf = choice([('categorical_crossentropy'), ('categorical_focal_crossentropy')])  # loss function
        self.op = choice([('adam'),('Nadam'),('adamax')])                  # optimization
        self.ac = 0                        # accuracy
        self.losss=np.inf

    def init_params(self):
        params = {'epochs': self.ep,
                  'activation1': self.a1,
                  'activation2': self.a2,
                  'activation3': self.a3,
                  'activation4': self.a4,
                  'pretrain': self.pretrain,
                  'dropout1': self.d1,
                  'unit1': self.u1,
                  'loss': self.lf,
                  'optimizer': self.op,
                  'accuracy': self.ac,
                  'losss': self.losss}
        return params


def init_net(p):
    return [Net() for _ in range(p)]


def fitness(n, n_c, x, y, b, x_test, y_test):
    a=[]
    l=[]
    f=[]
    for cnt, i in enumerate(n):
        p = i.init_params()
        ep = p['epochs']
        d1 = p['dropout1']
        u1 = p['unit1']
        a1 = p['activation1']
        a2 = p['activation2']
        a3 = p['activation3']
        a4 = p['activation4']
        pr = p['pretrain']
        lf = p['loss']
        op = p['optimizer']
        losss=p['losss']
        ac=p['accuracy']
        print('optimizer',op)
        print('activation1',a1)
        print('activation2',a2)
        print('dropout1',d1)
        print('number of neuron in unit1',u1)
        print('Loss function is :',lf )
        print("Pretraining model is: ", pr)
        print("The number of epochs is :", ep)


        try:                                # Parameter name    # Suggested value
            m = net_model(ep=ep,            # epoch number             12
                          a1=a1,            # activation 1           'relu'
                          a2=a2,            # activation 2           'relu'
                          a3=a3,            # activation 3           'relu'
                          a4=a4,            # activation 4           'softmax'
                          d1=d1,            # dropout 1                0.2
                          u1=u1,            # neuron number            256
                          op=op,            # optimizer               'adadelta'
                          lf=lf,            # loss function           'categorical crossentropy'
                          n_c=n_c,          # number of channel
                          x=x,              # train data
                          y=y,              # train label
                          b=b,              # bias value
                          x_test=x_test,    # test data
                          y_test=y_test,
                          pr = pr)    # test label

            # # Current best: 88%
            s = m.evaluate(x=[testb,testa], y=target_test, verbose=2)


            Y_pred = m.predict([testb,testa])
            y_pred = np.argmax(Y_pred, axis=1)
            target_test1 = np.argmax(target_test, axis=1)
            print('Confusion Matrix')
            (TP, FP, TN, FN) = perf_measure(target_test1, y_pred)
            print("The value for TP is: ", TP)
            print("The value for TN is: ", TN)
            print("The value for FP is: ", FP)
            print("The value for FN is: ", FN)
            print("The value for F1score is: ", f1_score(target_test1, y_pred, average='micro'))
            print(confusion_matrix(target_test1, y_pred))
            print('Classification Report')
            plt.show()
            i.ac = s[1]
            i.losss=s[0]

            print('Accuracy: {}'.format(i.ac * 100))
            print('loss: {}'.format(i.losss ))
            y_true0 = K.constant(target_test, dtype=tf.float32)
            y_pred0 = K.constant(Y_pred, dtype=tf.float32)
            print("***********************************")
            if i.losss>1:
              f1 = 0.1*i.ac + (0.9/i.losss)
            else:
              f1= 0.9*i.ac +(0.1/i.losss)

            f.append(f1)
            a.append(i.ac * 100)
            l.append(i.losss)
            print("The value for the fitness function is :", f)
            print("The accuracy  is :", f)
            print("The loss  is :", f)
        except Exception as e:
            print(e)
    best_ac=np.argmax(a)
    best_l=np.argmin(l)
    best_f=np.argmax(f)
    if best_f:
      n=n[best_f]
    elif best_ac==best_l:
      n=n[best_ac]
    else:
      n=n[best_l]
    print(best_ac,best_l)
    return [n]


early_stopping_cb =  tf.keras.callbacks.EarlyStopping(patience=50,restore_best_weights=True)
lr_scheduler1 = tf.keras.callbacks.ReduceLROnPlateau(factor=0.333, patience=50)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("Fused_bearing_fault.keras",save_best_only=True)
def net_model(ep, a1, a2, a3, a4, d1, u1, op, lf, n_c, x, y, b, x_test, y_test, pr):
    input_A = Input((9, 1))
    Conv1_out =  Conv1D(filters=1, strides =1,kernel_size=5, activation=a1,padding='same')(input_A)
    GRU_out = GRU(1, return_sequences=True, dropout=0.2, kernel_initializer=initializers.glorot_normal(seed=777),
                  bias_initializer='zeros')(input_A)
    Attention_out = Attention()([input_A, Conv1_out])
    Attention_out1 = Attention()([input_A, GRU_out])
    Concatenation = Concatenate()([Attention_out,Attention_out1])
    Concatenation = BatchNormalization()(Concatenation)
    Conv1_out1 =  Conv1D(filters=1, strides =1,kernel_size=5,activation=a1 ,padding='same')(Concatenation)
    GRU_out1 = GRU(1, return_sequences=True, dropout=0.2, kernel_initializer=initializers.glorot_normal(seed=777),
                   bias_initializer='zeros')(Concatenation)
    Concatenation2 = Concatenate()([Conv1_out1,GRU_out1])
    Concatenation2 = BatchNormalization()(Concatenation2)
    output_tensor, weights = MH_layer(Concatenation2, Concatenation,
                                   return_attention_scores=True)
    input_B = Input((224,224, 3))
    base_model = pr
    x = base_model(input_B, training=False)
    output_tensor_EEG = tf.keras.layers.GlobalAveragePooling2D()(x)
    flatten_Moca = Flatten()(output_tensor)
    flatten_EEG = Flatten()(output_tensor_EEG)
    Concatenation_total = Concatenate()([flatten_Moca,flatten_EEG])
    out1 = Dense(u1, activation=a3, kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros')(Concatenation_total)
    dropout1 = Dropout(d1)(out1)
    out = Dense(3, activation='softmax', kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros')(dropout1)
    model_Transfomer_Attention_fused = Model(inputs=[input_A,input_B], outputs=[out])


    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    model_Transfomer_Attention_fused.compile(optimizer=op, loss=lf, metrics=["accuracy"])
    print("Training has started")
    history = model_Transfomer_Attention_fused.fit([trainb,traina], target_train, batch_size=b, epochs=ep, verbose = 2, validation_split=0.2,
                                                   callbacks=[early_stopping_cb,lr_scheduler1,checkpoint_cb])
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
    plt.show()
    print("Training has finished")
    #model_Transfomer_Attention_fused.save("CNN_GA_Readmission.h5")
    return model_Transfomer_Attention_fused
#model with skip

def selection(n):
    n = sorted(n, key=lambda j: j.ac, reverse=True)
    #n = sorted(n, key=lambda j: j.losss, reverse=True)
    n = n[:2]
    return n


def crossover(n):
    offspring = []
    p1 = choice(n)
    p2 = choice(n)
    c1 = Net()
    c2 = Net()
    c1.ep = int(p2.ep) + 2
    c2.ep = int(p1.ep) + 2
    offspring.append(c1)
    offspring.append(c2)
    n.extend(offspring)
    return n
def crossover1(n):
    offspring = []
    p3 = choice(n)
    p4 = choice(n)
    c3 = Net()
    c4 = Net()
    c3.u1 = int(p4.u1) + 10
    c4.u1 = int(p3.u1) + 10
    offspring.append(c3)
    offspring.append(c4)
    n.extend(offspring)
    return n





def mutate(n):
    for i in n:
        if uniform(0, 1) <= 0.10991:
            i.ep += randint(100, 200)
            i.u1 += randint(10, 100)
    return n




In [ ]:
P = 2 # Population
G = 50 # Generation
B = 16 # Batch size
C = 3  # Class number
T = 0.964  # Threshold
N = init_net(p=P)  # Create population number networks
accuracy_list = []
for g in range(G):
    print('Generation {}'.format(g + 1))
    N = fitness(n=N,
                n_c=C,
                x=[trainb,traina],
                y=target_train,
                b=B,
                x_test=[testb,testa],
                y_test=target_test)
    N = selection(n=N)
    N = crossover(n=N)
    N = crossover1(n=N)
    N = mutate(n=N)
    for q in N:
        accuracy_list.append(q.ac * 100)
        if q.ac > T:
            print('Threshold satisfied')
            print(q.init_params())
            print('Best accuracy: {}%'.format(q.ac * 100))
            exit(code=0)
    print("The best accuracy so far {}%".format(max(accuracy_list)))

Generation 1
optimizer adam
activation1 leaky_relu
activation2 selu
dropout1 0.2
number of neuron in unit1 1523
Loss function is : categorical_focal_crossentropy
Pretraining model is:  <Functional name=densenet201, built=True>
The number of epochs is : 943
Training has started
Epoch 1/943
28/28 - 195s - 7s/step - accuracy: 0.3705 - loss: 0.4777 - val_accuracy: 0.3364 - val_loss: 2.6741 - learning_rate: 0.0010
Epoch 2/943
28/28 - 47s - 2s/step - accuracy: 0.4034 - loss: 0.1597 - val_accuracy: 0.4455 - val_loss: 0.1837 - learning_rate: 0.0010
Epoch 3/943
28/28 - 32s - 1s/step - accuracy: 0.4193 - loss: 0.1315 - val_accuracy: 0.4455 - val_loss: 0.2215 - learning_rate: 0.0010
Epoch 4/943
28/28 - 45s - 2s/step - accuracy: 0.4455 - loss: 0.1197 - val_accuracy: 0.2227 - val_loss: 0.1512 - learning_rate: 0.0010
Epoch 5/943
28/28 - 36s - 1s/step - accuracy: 0.4795 - loss: 0.1151 - val_accuracy: 0.4455 - val_loss: 0.1521 - learning_rate: 0.0010
Epoch 6/943
28/28 - 44s - 2s/step - accuracy: 0.521